# Registering your API Keys

In [ ]:
import os
from registered_keys import openapi_key

os.environ['OPENAI_API_KEY'] = openapi_key

# Components of a basic prompt

In [ ]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=1)
prediction = llm.invoke(
    """Suggest a weekend getaway place near the city of my choice.
    ###city: San Francisco
    ###Getaway place:""")
print(prediction)

## Making prompts more specific

In [ ]:
llm = OpenAI(temperature=0.9)
prediction = llm.invoke("Suggest a weekend getaway place near the city of my choice. Only output name of the place. ###city: San Francisco ###Getaway place:")
print(prediction)

# Prompt Templates

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables =['city'],
    template = """
    Suggest a weekend getaway place near the city of my choice. Only output name of the place.
    ###city: {city}
    ###Getaway place:"""
)
prompt = prompt_template.format(city = "San Francisco")

print(llm.predict(prompt))

## Adding multiple variables to the template

In [ ]:
prompt_template = PromptTemplate(
    input_variables =['city', 'days'],
    template = """
    Suggest a {days} day getaway place near the city of my choice. Only output name of the place.
    ###city: {city}
    ###Getaway place:"""
)
prompt = prompt_template.format(days=5, city="San Francisco")

print(llm.invoke(prompt))

# Chaining with LLMChains
## Example 1: Prompt chain with new prompt

In [ ]:
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain

destination_prompt_template = PromptTemplate(
    input_variables =['city'],
    template = "Suggest a weekend getaway place near the {city} of my choice. Only output name of the place. ###city: {city} ###Getaway place:"
)

destination_chain = LLMChain(llm=llm, prompt=destination_prompt_template)

itinerary_prompt_template = PromptTemplate(
    input_variables =['destination'],
    template = "Based on the {destination} and days suggest an itinerary"
)

itinerary_chain = LLMChain(llm=llm, prompt=itinerary_prompt_template)

chain = SimpleSequentialChain(chains = [destination_chain, itinerary_chain])

content = chain.invoke("San Francisco")
print(content)

## Example 2: How to take more than one input? A deeper dive

In [ ]:
from langchain.chains import SequentialChain

# Define the prompt to suggest a destination based on the city and number of days
destination_prompt = PromptTemplate(
    input_variables=["city", "days"],
    template="Suggest a {days} day getaway place near the {city} of my choice. Only output the name of the place. ###city: {city} ###Getaway place:"
)

# Create the destination chain to use the LLM for suggesting a destination
destination_chain = LLMChain(
    llm=llm,
    prompt=destination_prompt,
    output_key="destination"
)

# Define the prompt to generate an itinerary based on the destination and number of days
itinerary_prompt = PromptTemplate(
    input_variables=["destination", "days"],
    template="Suggest names of places to visit on each day for {days} days to {destination}."
)

# Create the itinerary chain to use the LLM for generating an itinerary
itinerary_chain = LLMChain(
    llm=llm,
    prompt=itinerary_prompt,
    output_key="itinerary"
)

# Combine the chains into a SequentialChain
e2e_chain = SequentialChain(
    chains=[destination_chain, itinerary_chain],
    input_variables=["city", "days"],
    output_variables=["destination", "itinerary"],
    verbose=True
)

# Function to plan the weekend getaway using the e2e chain
def plan_weekend_getaway(city, days):
    inputs = {"city": city, "days": days}
    result = e2e_chain.invoke(inputs)
    destination = result["destination"].strip()
    itinerary = result["itinerary"].strip()
    print(f"Suggested Destination: {destination}")
    print(f"Suggested Itinerary:\n{itinerary}")
    
plan_weekend_getaway("San Francisco", 5)

# Memory

In [ ]:


chain = LLMChain(llm=llm, prompt=destination_prompt_template)
name = chain.invoke("Pittsburgh")
print("memory: ", chain.memory)
print("memory type: ", type(chain.memory))



In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=destination_prompt_template, memory=memory)
destination = chain.invoke("New York")
print(destination)

destination = chain.invoke("New Delhi")
print(destination)

print(chain.memory.buffer)